## 10단계: 테스트

> 소프트웨어 개발에서는 테스트를 빼놓을 수 없습니다. \
테스트를 해야 실수(버그)를 예방할 수 있으며 테스트를 자동화해야 소프트웨어의 품질을 유지할 수 있습니다. \
DeZero도 마찬가지 입니다. \
이번 단계에서는 테스트 방법, 특히 딥러닝 프레임워크의 테스트 방법에 대해 설명하겠습니다.

### 10.1 파이썬 단위 테스트

파이썬의 단위 테스트 표준 라이브러리인 unittest를 사용해보자.

In [1]:
# 필요 모듈 정의

import numpy as np

class Variable:
    def __init__(self, data):
        if not isinstance(data, (type(None), np.ndarray)):
            raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.asarray(x)
    return x

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

In [ ]:
import unittest

class SquareTest(unittest.TestCase):
    
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

# jupyter notebook 전용 unittest 실행 코드
#     argv=['first-arg-is-ignored'] 는 jupyter가 전달하는 자체 인자를 무시하기 위함
#     exit=False 는 테스트 종료 후 커널 종료 방지
unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


### 10.2 square 함수의 역전파 테스트

In [ ]:
class SquareTest(unittest.TestCase):
    
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
    
    # 역전파 테스트 추가
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)


unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.010s

OK


### 10.3 기울기 확인을 이용한 자동 테스트

미분의 기댓값을 수기 입력하지 말고 **기울기 확인(gradient checking)**으로 자동화하자.

기울기 확인은 수치 미분 결과와 역전파 결과를 비교하여 차이가 크면 역전파 구현에 문제가 있다고 판단하는 검증 기법이다.

In [5]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)


class SquareTest(unittest.TestCase):
    
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
    
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)
    
    # 기울기 확인을 이용한 자동 테스트트
    def test_gradient_check(self):
        x = Variable(np.random.rand(1))  # 0-1 사이 랜덤값 생성
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)


unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.078s

OK


이 때 얼마나 가까운지에 대한 판단을 `np.allclose(a, b, rtol=1e-5, atol=1e-8)`과 같이 인수 `rtol`과 `atol`로 지정할 수 있다.

이 함수는 `a`와 `b`의 모든 요소가 다음 조건을 만족하면 `True`를 반환한다.

`|a - b| <= (atol + rtol * |b|)`

따라서 `atol`과 `rtol` 값을 미세하게 조정해야 할 수도 있다.

기준을 정하는 데는 [참고문헌 [5]](https://cs231n.github.io/neural-networks-3/, "Fei-Fei Li, et al. \"CS231n: Convolutional Neural Networks for Visual Recognition.\"") 등이 도움된다.

### 10.4 테스트 정리

DeZero 개발에 한정한다면 테스트에 관한 지식은 이 정도면 충분할 것이다.

앞으로 책에서는 테스트를 생략하지만, 필요하다면 스스로 추가해보기 바란다.

또한 테스트 파일들은 하나의 장소에서 관리하는 것이 바람직하기 때문에, 책에서도 테스트 코드는 `tests` 디렉터리에 모아두었다.

관심있는 사람들은 찾아보길 바란다.

**제1고지 완료.**